# 检测晕影（Vignetting）
晕影通常表现为图像边缘的暗角，可以通过分析图像边缘与中心的亮度差异来检测。

In [1]:
import cv2
import numpy as np

In [2]:
def cv_imread(file_path):
    """读取带中文路径的图片文件
    Args:
        file_path (_type_): _description_
    Returns:
        _type_: _description_
    """
    return cv2.imdecode(np.fromfile(file_path, dtype=np.uint8), -1)


def cv_imwrite(file_path, img):
    """保存带中文路径的图片文件
    Args:
        file_path (_type_): _description_
        img (_type_): _description_
    """
    cv2.imencode(".png", img)[1].tofile(file_path)

In [3]:
# 读取图像并转换为灰度
img = cv_imread(r'D:\染色体测试数据\230111-二值化测试\刘老师数据\=好1=4.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img

In [4]:
# 亮度分析:分析图像的亮度分布。晕影通常会导致图像边缘区域的亮度低于中心区域。
rows, cols = gray.shape
center_x, center_y = int(cols / 2), int(rows / 2)
radius = min(center_x, center_y)

# 初始化亮度分布数组
luminance_distribution = []

# 分析从中心到边缘的亮度分布
for r in range(0, radius, 10):
    mask = np.zeros_like(gray, dtype=np.uint8)
    cv2.circle(mask, (center_x, center_y), r, 255, -1)
    mean_val = cv2.mean(gray, mask=mask)[0]
    luminance_distribution.append(mean_val)

In [12]:
luminance_distribution

[255.0,
 254.90220820189273,
 254.88305489260145,
 254.8840836582772,
 254.74766169154228,
 252.26921606118546,
 248.04119053946317,
 244.67137188577377,
 244.1172750361038,
 245.22829632540774,
 246.52691218130315,
 247.3969089808062,
 247.3926589275843,
 246.13226067788307,
 245.37840693006552,
 245.0071872214598,
 244.89783655341435,
 245.12070275926638,
 245.43452070947774,
 245.6529121717577,
 246.07960741548524,
 246.33179307606224,
 246.28994230756578,
 245.78203346389185,
 245.2425145232344,
 245.18798803999573,
 245.03411707643264,
 244.86491219642147,
 245.00821918920786,
 245.18629466026218,
 245.29541876991973,
 245.04478407912737,
 244.81619240370955,
 245.08747445750518,
 245.4672238362087,
 245.66741257650773,
 245.675070564739,
 245.28929862701563,
 244.9220923234296,
 245.09904545844574,
 245.40895896543148,
 245.66324839739036,
 245.82055645689599,
 245.91169923196512,
 246.01581416303156,
 246.12812813914334,
 246.2525330023694,
 246.2996528383241,
 246.2773986561639

In [5]:
# 使用图像亮度梯度的平均值或中位数作为阈值设定的基准是一种智能化的方法，可以根据图像的具体特征动态调整阈值。
# 计算亮度梯度: 首先，你需要计算图像的亮度梯度。这通常涉及到计算图像每个像素点的亮度变化率。
# 使用Sobel算子计算X和Y方向上的梯度
grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

# 计算梯度的综合幅度
grad_magnitude = cv2.magnitude(grad_x, grad_y)

In [13]:
grad_magnitude

array([[0.        , 2.        , 2.        , ..., 0.        , 2.        ,
        0.        ],
       [2.        , 3.16227766, 4.24264069, ..., 1.41421356, 3.16227766,
        4.        ],
       [4.        , 2.82842712, 3.16227766, ..., 4.24264069, 4.47213595,
        6.        ],
       ...,
       [2.        , 3.16227766, 1.41421356, ..., 2.        , 7.07106781,
        2.        ],
       [2.        , 2.        , 2.82842712, ..., 5.09901951, 3.16227766,
        2.        ],
       [0.        , 0.        , 2.        , ..., 0.        , 0.        ,
        0.        ]])

In [6]:
# 计算梯度的统计量: 接下来，计算梯度幅度的平均值或中位数。这将作为阈值设定的基准。
# 计算平均值和中位数
mean_grad = np.mean(grad_magnitude)
median_grad = np.median(grad_magnitude)

In [14]:
mean_grad, median_grad

(14.895418370106777, 6.0)

In [7]:
# 设定阈值: 根据平均值或中位数来设定阈值。你可以直接使用这些值，或者根据需要对它们进行调整（比如乘以一个系数）。
# 设定阈值，可以根据需要调整这个系数
threshold_factor = 0.5
threshold = mean_grad * threshold_factor

In [10]:
threshold

7.447709185053388

In [8]:
# 检测晕影: 晕影检测可以通过分析亮度分布的梯度来完成。如果亮度从中心到边缘急剧下降，可能表明存在晕影。
# 计算亮度梯度
luminance_gradient = np.gradient(luminance_distribution)

# 检测亮度梯度是否显著下降
vignette_detected = any(gradient < -threshold for gradient in luminance_gradient)

In [11]:
luminance_gradient

array([-0.0977918 , -0.05847255, -0.00906227, -0.0676966 , -1.3074338 ,
       -3.35323558, -3.79892209, -1.96195775,  0.27846222,  1.20481857,
        1.08430633,  0.43287337, -0.63232415, -1.007126  , -0.56253673,
       -0.24028519,  0.05675777,  0.26834208,  0.26610471,  0.32254335,
        0.33944045,  0.10516745, -0.27487981, -0.52371389, -0.29702271,
       -0.10419872, -0.16153792, -0.01294894,  0.16069123,  0.14359979,
       -0.07075529, -0.23961318,  0.02134519,  0.32551572,  0.28996906,
        0.10392336, -0.18905697, -0.37648912, -0.09512658,  0.24343332,
        0.28210147,  0.20579875,  0.12422542,  0.09762885,  0.10821445,
        0.11835942,  0.08576235,  0.01243283, -0.02390712, -0.02679876,
        0.04798709,  0.14834043,  0.14292677,  0.11023304,  0.12784076,
        0.13648571,  0.12790634,  0.14683742,  0.16514992,  0.15432424,
        0.11477297,  0.08891968,  0.1028984 ,  0.13239674,  0.14354376,
        0.13727135,  0.13437902,  0.12778833,  0.11028261,  0.10

In [9]:
# 结果: 根据检测结果，可以判断图像是否存在晕影。
if vignette_detected:
    print("晕影检测到了")
else:
    print("未检测到晕影")

未检测到晕影
